In [35]:
import pandas as pd

df = pd.read_csv('outliers_data_test.csv', encoding='cp949')
len(df)

19

In [20]:
api_key = ''
df = pd.read_csv('outliers_data2.csv')

In [58]:
import pandas as pd
import google.generativeai as genai
import re

def create_prompt(df, start_index, chunk_size=100):
    prompt = "다음 텍스트와 감정 쌍을 분석하고, 각 텍스트가 해당 감정과 일치하는지 확인해주세요. 일치하지 않는 경우 적절한 감정을 제안해주세요.\n\n"
    for i in range(start_index, min(start_index + chunk_size, len(df))):
        prompt += f"{i+1}. 텍스트: {df.loc[i, 'text']}\n   감정: {df.loc[i, 'emotion']}\n\n"
        
    prompt += "보기의 감정으로만 답변해주세요.:\n"
    prompt += "emotions = ['당황', '분노', '불안', '슬픔', '중립', '행복'] \n"

    prompt += "각 항목에 대해 다음 형식으로 답변해주세요:\n"
    prompt += "항목 번호: [일치 여부] (일치/불일치) - [제안된 감정(불일치 경우에만)]\n"
    return prompt

def process_gemini_response(response_text, df, start_index):
    lines = response_text.split('\n')
    for line in lines:
        match = re.match(r'(\d+): (\w+) - (\w+)', line)
        if match:
            item_num, match_status, suggested_emotion = match.groups()
            idx = start_index + int(item_num) - 1
            if idx < len(df) and match_status == '불일치':
                df.loc[idx, 'emotion'] = suggested_emotion

def analyze_emotions(df, api_key):
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-pro')

    for start_index in range(0, len(df), 20):
        prompt = create_prompt(df, start_index)
        
        response = model.generate_content(prompt)
        print(response)
        
        if response.text:
            process_gemini_response(response.text, df, start_index)
        else:
            print(f"Warning: Empty response for prompt starting at index {start_index}")

    return df

# 메인 실행 코드
if __name__ == "__main__":

    # 감정 분석 실행
    updated_df = analyze_emotions(df, api_key)

    # 결과 저장
    updated_df.to_csv('updated_emotion_data2.csv', index=False)
    print("감정 분석이 완료되었습니다. 결과가 'updated_emotion_data.csv'에 저장되었습니다.")

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "1: \uc77c\uce58 - \ubd88\uc548\n2: \uc77c\uce58 - \ubd88\uc548\n3: \uc77c\uce58 - \uacb0\uc758\n4: \uc77c\uce58 - \ubd84\ub178\n5: \uc77c\uce58 - \uae30\uc068\n6: \uc77c\uce58 - \uc9dc\uc99d\n7: \uc77c\uce58 - \ud638\uae30\uc2ec\n8: \uc77c\uce58 - \uc5fc\ub824\n9: \uc77c\uce58 - \uc131\uae09\ud568\n10: \uc77c\uce58 - \uae34\uc7a5\n11: \uc77c\uce58 - \uae30\uc068\n12: \uc77c\uce58 - \uae30\uc068\n13: \ubd88\uc77c\uce58 - \uc815\ubcf4\uc801\n14: \uc77c\uce58 - \uae30\uc068\n15: \uc77c\uce58 - \uac71\uc815\n16: \uc77c\uce58 - \uc2e4\ub9dd\n17: \uc77c\uce58 - \uacb0\uc758\n18: \uc77c\uce58 - \uae30\uc068\n19: \uc77c\uce58 - \uc548\uc2ec"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 

In [54]:
def process_gemini_response(response_text, df, start_index):
    lines = response_text.split('\n')
    print(lines)
    for line in lines:
        print(line)
        match = re.match(r'(\d+): \[(\w+)\]( - \[(\w+)\])?', line)
        print(match)
        if match:
            print("match",match)
            item_num, match_status, _, suggested_emotion = match.groups()
            idx = start_index + int(item_num) - 1
            if idx < len(df) and match_status == '불일치' and suggested_emotion:
                df.loc[idx, 'emotion'] = suggested_emotion

def process_gemini_response(response_text, df, start_index):
    lines = response_text.split('\n')
    for line in lines:
        match = re.match(r'(\d+): (\w+) - (\w+)', line)
        if match:
            item_num, match_status, suggested_emotion = match.groups()
            idx = start_index + int(item_num) - 1
            if idx < len(df) and match_status == '불일치':
                df.loc[idx, 'emotion'] = suggested_emotion


In [44]:
response.text

'1: 불일치 - 두려움\n2: 불일치 - 불안\n3: 불일치 - 결의\n4: 불일치 - 분노\n5: 불일치 - 기쁨\n6: 불일치 - 짜증\n7: 불일치 - 호기심\n8: 불일치 - 염려\n9: 불일치 - 성급함\n10: 불일치 - 긴장\n11: 불일치 - 기쁨\n12: 불일치 - 기쁨\n13: 불일치 - 정보적\n14: 불일치 - 기쁨\n15: 불일치 - 걱정\n16: 불일치 - 실망\n17: 불일치 - 결의\n18: 불일치 - 기쁨\n19: 불일치 - 안심'